# <h1><center>Geocoding</center></h1>



<img src="https://geopy.readthedocs.io/en/stable/_static/geopy_and_geocoding_services.svg"></img>

In [2]:
import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import folium
from folium.plugins import FastMarkerCluster

### Geocoding Single Address 

In [3]:

locator = Nominatim(user_agent="myGeocoder")
location = locator.geocode("Champ de Mars, Paris, France")

In [4]:
print(location.address)
print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))


Champ de Mars, Place Edwige Feuillère, Quartier du Gros-Caillou, Paris, Île-de-France, France métropolitaine, 75007, France
Latitude = 48.85614465, Longitude = 2.297820393322227


### Geocoding with Pandas DataFrame

In [8]:
df = pd.read_csv("addresses.csv")
df.head()

,Typ,Nr,Namn,Address1,Address3,Address4,Address5,Telefon
0,Butik,102,Fältöversten,Karlaplan 13,115 20,STOCKHOLM,Stockholms län,08/662 22 89
1,Butik,104,NaN,Nybrogatan 47,114 39,STOCKHOLM,Stockholms län,08/662 50 16
2,Butik,106,Garnisonen,Karlavägen 100 A,115 26,STOCKHOLM,Stockholms län,08/662 64 85
3,Butik,110,NaN,Hötorgshallen,111 57,STOCKHOLM,Stockholms län,08/56849241
4,Butik,113,Sergel,Drottninggatan 45,111 21,STOCKHOLM,Stockholms län,08/21 47 44


In [9]:
df['ADDRESS'] = df['Address1'].astype(str) + ',' + \
                df['Address3'] + ',' + \
                df['Address4'] + ',' + \
                df['Address5'] + ',' + ' Sweden'   

df.head()

,Typ,Nr,Namn,Address1,Address3,Address4,Address5,Telefon,ADDRESS
0,Butik,102,Fältöversten,Karlaplan 13,115 20,STOCKHOLM,Stockholms län,08/662 22 89,"Karlaplan 13,115 20,STOCKHOLM,Stockholms län, ..."
1,Butik,104,NaN,Nybrogatan 47,114 39,STOCKHOLM,Stockholms län,08/662 50 16,"Nybrogatan 47,114 39,STOCKHOLM,Stockholms län,..."
2,Butik,106,Garnisonen,Karlavägen 100 A,115 26,STOCKHOLM,Stockholms län,08/662 64 85,"Karlavägen 100 A,115 26,STOCKHOLM,Stockholms l..."
3,Butik,110,NaN,Hötorgshallen,111 57,STOCKHOLM,Stockholms län,08/56849241,"Hötorgshallen,111 57,STOCKHOLM,Stockholms län,..."
4,Butik,113,Sergel,Drottninggatan 45,111 21,STOCKHOLM,Stockholms län,08/21 47 44,"Drottninggatan 45,111 21,STOCKHOLM,Stockholms ..."


In [10]:
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
df['location'] = df['ADDRESS'].apply(geocode)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [11]:
df.head()


,Typ,Nr,Namn,Address1,Address3,Address4,Address5,Telefon,ADDRESS,location,point
0,Butik,102,Fältöversten,Karlaplan 13,115 20,STOCKHOLM,Stockholms län,08/662 22 89,"Karlaplan 13,115 20,STOCKHOLM,Stockholms län, ...","(Karlaplan, Vedbäraren, Östermalm, Östermalms ...","(59.3383148, 18.0899599, 0.0)"
1,Butik,104,NaN,Nybrogatan 47,114 39,STOCKHOLM,Stockholms län,08/662 50 16,"Nybrogatan 47,114 39,STOCKHOLM,Stockholms län,...","(47, Nybrogatan, Brunfisken, Östermalm, Österm...","(59.3372072, 18.0790982, 0.0)"
2,Butik,106,Garnisonen,Karlavägen 100 A,115 26,STOCKHOLM,Stockholms län,08/662 64 85,"Karlavägen 100 A,115 26,STOCKHOLM,Stockholms l...","(Karlavägen, Infanteristen, Östermalm, Österma...","(59.3353799, 18.1006258, 0.0)"
3,Butik,110,NaN,Hötorgshallen,111 57,STOCKHOLM,Stockholms län,08/56849241,"Hötorgshallen,111 57,STOCKHOLM,Stockholms län,...","(Hötorgshallen, Hötorget, Konserthuset, Norrma...","(59.3343273, 18.0626039, 0.0)"
4,Butik,113,Sergel,Drottninggatan 45,111 21,STOCKHOLM,Stockholms län,08/21 47 44,"Drottninggatan 45,111 21,STOCKHOLM,Stockholms ...","(Drottninggatan, Torgvågen, Norrmalm, Norrmalm...","(59.3336068, 18.0616879, 0.0)"


In [12]:
# split point column into latitude, longitude and altitude columns
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)
df.head()

,Typ,Nr,Namn,Address1,Address3,Address4,Address5,Telefon,ADDRESS,location,point,latitude,longitude,altitude
0,Butik,102,Fältöversten,Karlaplan 13,115 20,STOCKHOLM,Stockholms län,08/662 22 89,"Karlaplan 13,115 20,STOCKHOLM,Stockholms län, ...","(Karlaplan, Vedbäraren, Östermalm, Östermalms ...","(59.3383148, 18.0899599, 0.0)",59.338315,18.089960,0.0
1,Butik,104,NaN,Nybrogatan 47,114 39,STOCKHOLM,Stockholms län,08/662 50 16,"Nybrogatan 47,114 39,STOCKHOLM,Stockholms län,...","(47, Nybrogatan, Brunfisken, Östermalm, Österm...","(59.3372072, 18.0790982, 0.0)",59.337207,18.079098,0.0
2,Butik,106,Garnisonen,Karlavägen 100 A,115 26,STOCKHOLM,Stockholms län,08/662 64 85,"Karlavägen 100 A,115 26,STOCKHOLM,Stockholms l...","(Karlavägen, Infanteristen, Östermalm, Österma...","(59.3353799, 18.1006258, 0.0)",59.335380,18.100626,0.0
3,Butik,110,NaN,Hötorgshallen,111 57,STOCKHOLM,Stockholms län,08/56849241,"Hötorgshallen,111 57,STOCKHOLM,Stockholms län,...","(Hötorgshallen, Hötorget, Konserthuset, Norrma...","(59.3343273, 18.0626039, 0.0)",59.334327,18.062604,0.0
4,Butik,113,Sergel,Drottninggatan 45,111 21,STOCKHOLM,Stockholms län,08/21 47 44,"Drottninggatan 45,111 21,STOCKHOLM,Stockholms ...","(Drottninggatan, Torgvågen, Norrmalm, Norrmalm...","(59.3336068, 18.0616879, 0.0)",59.333607,18.061688,0.0


In [13]:
df.columns

Index(['Typ', 'Nr', 'Namn', 'Address1', 'Address3', 'Address4', 'Address5',
       'Telefon', 'ADDRESS', 'location', 'point', 'latitude', 'longitude',
       'altitude'],
      dtype='object')

In [14]:
df = df.drop(['Address1', 'Address3', 'Address4', 'Address5','Telefon', 'ADDRESS', 'location', 'point'], axis=1)

In [15]:
df.head()

,Typ,Nr,Namn,latitude,longitude,altitude
0,Butik,102,Fältöversten,59.338315,18.089960,0.0
1,Butik,104,NaN,59.337207,18.079098,0.0
2,Butik,106,Garnisonen,59.335380,18.100626,0.0
3,Butik,110,NaN,59.334327,18.062604,0.0
4,Butik,113,Sergel,59.333607,18.061688,0.0


In [18]:
df.latitude.isnull().sum()

0

In [17]:
df = df[pd.notnull(df["latitude"])]

In [19]:
map1 = folium.Map(
    location=[59.338315,18.089960],
    tiles='cartodbpositron',
    zoom_start=12,
    
)

In [20]:
df.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]]).add_to(map1), axis=1)
map1

In [ ]:
#map1.save("map1.html")

In [21]:

folium_map = folium.Map(location=[59.338315,18.089960],
                        zoom_start=12,
                        tiles='CartoDB dark_matter')


FastMarkerCluster(data=list(zip(df['latitude'].values, df['longitude'].values))).add_to(folium_map)
folium.LayerControl().add_to(folium_map)
folium_map

In [ ]:
#folium_map.save("map2.html")